transforms.Compose()はpytorchで画像を指定したデータ形式に変換するためのメソッドです。ToTensor()を使用することで、取り込んだMNIST画像（最初はnumpy配列）をテンソルへと変換します。

transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))で各チャンネルにおける、テンソル（画像）の輝度値を平均0.5、標準偏差0.5になるように標準化します。今だったら3チャンネル分をまとめて指定します。

最初にdiscriminator.zero_grad()で勾配の初期値を0に設定します。その後は。トレイン画像とgeneratorが生成したフェイク画像それぞれについて損失を定義し、スコアを出力します。

それぞれ別々にスコアを計算しなければいけない理由は、前回のテキストのLossをみてもらえればわかると思います。Lossの式が第一項と第二項に別れているのはそのためです。

backward()メソッドはpytorchのtorch.Tensorの標準的なクラスであり、backward()を呼び出すだけで全ての勾配を自動的に計算させることが可能です。そしてstepメソッドで更新させ

In [17]:
# 保存用のdirectory作成
#!mkdir results

In [1]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torch.nn as nn
from torch.autograd import Variable
import itertools
import math
import time
import matplotlib.pyplot as plt
from IPython import display
import torchvision.datasets as dsets
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,),(0.5,))
])

train_dataset = dsets.MNIST(root='~/tmp/mnis', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)



class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        out = self.model(x.view(x.size(0), 784))
        out = out.view(out.size(0), -1)
        return out



class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
        nn.Linear(100,256),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Dropout(0.3),
        nn.Linear(256, 512),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Dropout(0.3),
        nn.Linear(512, 1024),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Dropout(0.3),
        nn.Linear(1024, 784),
        nn.Tanh()
        )
        
    def forward(self, x):
        x = x.view(x.size(0),100)
        out = self.model(x) # 100,784
        return out
        
discriminator = Discriminator()
generator = Generator()



criterion = nn.BCELoss()
lr = 0.0002
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)



def train_discriminator(discriminator, images, real_labels, fake_images, fake_labels):
    discriminator.zero_grad()
    outputs = discriminator(images)
    real_loss = criterion(outputs, real_labels)
    real_score = outputs
    
    outputs = discriminator(fake_images) 
    fake_loss = criterion(outputs, fake_labels)
    fake_score = outputs

    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss, real_score, fake_score

def train_generator(generator, real_labels, outputs):
    # 勾配を0に設定
    generator.zero_grad()
    # lossを設定
    g_loss = criterion(outputs, real_labels)
    # backwardを設定
    g_loss.backward()
    # optimizerを設定
    g_optimizer.step()
    # generatorのlossを返す
    return g_loss

# draw samples from the input distribution to inspect the generation on training 
num_test_samples = 16
test_noise = Variable(torch.randn(num_test_samples, 100))

# create figure for plotting
size_figure_grid = int(math.sqrt(num_test_samples))
fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(6, 6))
for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
    ax[i,j].get_xaxis().set_visible(False)
    ax[i,j].get_yaxis().set_visible(False)

# set number of epochs and initialize figure counter
num_epochs = 200
num_batches = len(train_loader)
num_fig = 0


for epoch in range(num_epochs):
    for n, (images, _) in enumerate(train_loader):
        images = Variable(images)
        real_labels = Variable(torch.ones(images.size(0)))
        
        # Sample from generator
        noise = Variable(torch.randn(images.size(0), 100))
        fake_images = generator(noise)
        fake_labels = Variable(torch.zeros(images.size(0)))
        
        # Train the discriminator
        d_loss, real_score, fake_score = train_discriminator(discriminator, images, real_labels, fake_images, fake_labels)
        
        # Sample again from the generator and get output from discriminator
        noise = Variable(torch.randn(images.size(0), 100))
        fake_images = generator(noise)
        outputs = discriminator(fake_images)

        # Train the generator
        g_loss = train_generator(generator,real_labels, outputs)
        
        if (n+1) % 100 == 0:
            test_images = generator(test_noise)
            
            for k in range(num_test_samples):
                i = k//4
                j = k%4
                ax[i,j].cla()
                ax[i,j].imshow(test_images[k,:].data.cpu().numpy().reshape(28, 28), cmap='Greys')
            display.clear_output(wait=True)
            display.display(plt.gcf())
            
            plt.savefig('results/mnist-gan-%03d.png'%num_fig)
            num_fig += 1
            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, g_loss: %.4f, ' 
                  'D(x): %.2f, D(G(z)): %.2f' 
                  %(epoch + 1, num_epochs, n+1, num_batches, d_loss.data.item(), g_loss.data.item(),
                    real_score.data.mean(), fake_score.data.mean()))

fig.close()

<Figure size 600x600 with 16 Axes>

Epoch [200/200], Step[600/600], d_loss: 0.9687, g_loss: 1.5315, D(x): 0.68, D(G(z)): 0.35


AttributeError: 'Figure' object has no attribute 'close'

3.問題2 論文調査

GANは生成的モデルを訓練する新しい方法として導入された.GANはデータ分布を捉える生成モデルGと、標本データがGではなく学習データから得られる確率を推定する判別モデルDの2つの「敵対的」モデルで構成されている.
Gはデータxの分布を学習するためにノイズ分布からデータ空間の写像関数を学習する.
GとDの両方とも、多層パーセプトロンのような非線形写像関数である.
 GとDは両方とも同時に訓練されます.
G（1 - D（G（z）））を最小にするようにGのパラメータを調整し、L（D（X））を最小にするようにDのパラメータを調整する.それらが価値関数V（G、D）を持つ2人用のミニマックスゲームに従っているかのように見える.

GANのG,Dともに追加の情報yを入力することでconditionalモデルに拡張することができる.yは、クラスラベルや他のモダリティからのデータなど,あらゆる種類の補助情報とする.

One-Hot-Vectorとしてencodeされた,クラスラベルで条件付けをしたMNIST画像についてconditional敵対ネットを訓練した。ノイズは次元数100の一様分布.zとyは隠れ層にマッピングされ、ReLuの後,層数はそれぞれ200と1000である.次にマッピングする前にcombineして1200にする.最後にsigmoidを使用し784次元のMNISTをサンプル生成する.

Dはxを240単位と5個のmaxout [6]層に、yを50単位と5個のmaxout層にマッピングする.
両方の隠れ層は、シグモイド層に供給される前に,240単位と4個のジョイントマックスアウト層にマッピングされた.
ただし,Dは機能する場合,アーキテクチャはこの限りでなく,体感的にマックスアウトが良好であった.

サイズ100のミニバッチと初期学習率0.1の確率的勾配降下法を使用して訓練した.momentumは初期値0.5で0.7まで増加した.Dropoutは0.5でGとD両方に適用した.検証セットでの対数尤度の最善の推定値を停止点として使用した.

問題2で調査した、ConditionalGANを実装

'''
通常のGANのコード
'''

from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            
            
            # Generate a batch of new images
            gen_imgs = self.generator.predict([noise, )

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()



gan = GAN()
gan.train(epochs=30000, batch_size=32,sample_interval=200)

In [2]:
#!mkdir images2

In [47]:
# yを入力として追加するConditionalGANへの拡張
from __future__ import print_function, division
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt

import sys

import numpy as np
from keras import layers
from keras import backend as K

class ConditionalGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.y_size = 10

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        # labelを加える
        y = Input(shape=(self.y_size,))
        img = self.generator([z,y])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator([img, y])

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([z,y], validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        noise = Input(shape=(self.latent_dim,))
        # zは200yは1000
        z = layers.Dense(200, activation='relu')(noise)
        y_onehot = Input(shape=(self.y_size,))
        y = layers.Dense(1000, activation='relu')(y_onehot)
        gen_input = layers.concatenate([z, y],axis=1)
        dense_1 = Dense(256, activation=LeakyReLU(alpha=0.2))(gen_input)
        batch_1 = BatchNormalization(momentum=0.8)(dense_1)
        dense_2 = Dense(512, activation=LeakyReLU(alpha=0.2))(batch_1)
        batch_2 = BatchNormalization(momentum=0.8)(dense_2)
        dense_3 = Dense(1024, activation=LeakyReLU(alpha=0.2))(batch_2)
        batch_3 = BatchNormalization(momentum=0.8)(dense_3)
        dense_4 = Dense(np.prod(self.img_shape), activation='tanh')(batch_3)
        img = Reshape((self.img_shape))(dense_4)
        model = Model([noise, y_onehot], img)
        
        return model

    def build_discriminator(self):
        
        # xは240, yは50
        img = Input(shape=self.img_shape)
        flat = Flatten()(img)
        x = layers.Dense(240, activation='relu')(flat)
        y_onehot = Input(shape=(self.y_size,))
        y = layers.Dense(50, activation='relu')(y_onehot)
        dis_input = layers.concatenate([x,y],axis=1)
        dense_1 = Dense(512,activation=LeakyReLU(alpha=0.2))(dis_input)
        dense_2 = Dense(256,activation=LeakyReLU(alpha=0.2))(dense_1)
        validity = Dense(1,activation='sigmoid')(dense_2)
        
        #validity = (dis_input)
        model = Model([img,y_onehot], validity)
        return model

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)
        enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
        labels = enc.fit_transform(y_train[:, np.newaxis])
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            y_batch = labels[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
        
            gen_imgs = self.generator.predict([noise, y_batch])

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch([imgs,y_batch], valid)
            d_loss_fake = self.discriminator.train_on_batch([gen_imgs,y_batch], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch([noise,y_batch], valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        
        labels = np.zeros((r*c,10))
        y = np.random.randint(0,10,r*c)
        for i, v in enumerate(y):
            labels[i][v]=1
        gen_imgs = self.generator.predict([noise, labels])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images2/%d.png" % epoch)
        plt.close()



gan = ConditionalGAN()
gan.train(epochs=30000, batch_size=32,sample_interval=200)

C:\Users\umini\Anaconda3\envs\pytorch\lib\site-packages\keras\activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


(32, 10)


C:\Users\umini\Anaconda3\envs\pytorch\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.674841, acc.: 43.75%] [G loss: 0.447436]
(32, 10)
1 [D loss: 0.612832, acc.: 50.00%] [G loss: 0.496744]
(32, 10)
2 [D loss: 0.493885, acc.: 54.69%] [G loss: 0.653050]
(32, 10)
3 [D loss: 0.393715, acc.: 76.56%] [G loss: 0.874106]
(32, 10)
4 [D loss: 0.314070, acc.: 87.50%] [G loss: 1.064406]
(32, 10)
5 [D loss: 0.231851, acc.: 98.44%] [G loss: 1.295273]
(32, 10)
6 [D loss: 0.190180, acc.: 100.00%] [G loss: 1.584211]
(32, 10)
7 [D loss: 0.165483, acc.: 100.00%] [G loss: 1.678020]
(32, 10)
8 [D loss: 0.141075, acc.: 100.00%] [G loss: 1.887785]
(32, 10)
9 [D loss: 0.102382, acc.: 100.00%] [G loss: 2.085511]
(32, 10)
10 [D loss: 0.093606, acc.: 100.00%] [G loss: 2.204224]
(32, 10)
11 [D loss: 0.077771, acc.: 100.00%] [G loss: 2.284545]
(32, 10)
12 [D loss: 0.069796, acc.: 100.00%] [G loss: 2.475284]
(32, 10)
13 [D loss: 0.052115, acc.: 100.00%] [G loss: 2.569031]
(32, 10)
14 [D loss: 0.045938, acc.: 100.00%] [G loss: 2.769799]
(32, 10)
15 [D loss: 0.045894, acc.: 100.00%] [G l

126 [D loss: 0.513030, acc.: 79.69%] [G loss: 2.831076]
(32, 10)
127 [D loss: 0.288397, acc.: 90.62%] [G loss: 2.654761]
(32, 10)
128 [D loss: 0.156647, acc.: 93.75%] [G loss: 3.388905]
(32, 10)
129 [D loss: 0.198858, acc.: 92.19%] [G loss: 3.369594]
(32, 10)
130 [D loss: 0.327791, acc.: 85.94%] [G loss: 2.755102]
(32, 10)
131 [D loss: 0.340267, acc.: 84.38%] [G loss: 3.354048]
(32, 10)
132 [D loss: 0.225665, acc.: 93.75%] [G loss: 3.298706]
(32, 10)
133 [D loss: 0.144486, acc.: 96.88%] [G loss: 3.356095]
(32, 10)
134 [D loss: 0.282963, acc.: 89.06%] [G loss: 3.432322]
(32, 10)
135 [D loss: 0.484161, acc.: 76.56%] [G loss: 2.755918]
(32, 10)
136 [D loss: 0.171246, acc.: 93.75%] [G loss: 3.478739]
(32, 10)
137 [D loss: 0.224773, acc.: 93.75%] [G loss: 2.695714]
(32, 10)
138 [D loss: 0.209790, acc.: 90.62%] [G loss: 3.186520]
(32, 10)
139 [D loss: 0.101215, acc.: 98.44%] [G loss: 2.805155]
(32, 10)
140 [D loss: 0.330077, acc.: 85.94%] [G loss: 2.951848]
(32, 10)
141 [D loss: 0.300389, ac

...

9955 [D loss: 0.650803, acc.: 67.19%] [G loss: 0.931093]
(32, 10)
9956 [D loss: 0.637062, acc.: 60.94%] [G loss: 0.868807]
(32, 10)
9957 [D loss: 0.629012, acc.: 65.62%] [G loss: 0.838571]
(32, 10)
9958 [D loss: 0.726127, acc.: 48.44%] [G loss: 0.861383]
(32, 10)
9959 [D loss: 0.718433, acc.: 48.44%] [G loss: 0.881288]
(32, 10)
9960 [D loss: 0.693833, acc.: 48.44%] [G loss: 0.904767]
(32, 10)
9961 [D loss: 0.658753, acc.: 65.62%] [G loss: 0.860207]
(32, 10)
9962 [D loss: 0.663140, acc.: 56.25%] [G loss: 0.930364]
(32, 10)
9963 [D loss: 0.632657, acc.: 64.06%] [G loss: 0.884247]
(32, 10)
9964 [D loss: 0.631009, acc.: 64.06%] [G loss: 0.868534]
(32, 10)
9965 [D loss: 0.679882, acc.: 60.94%] [G loss: 0.864402]
(32, 10)
9966 [D loss: 0.653146, acc.: 53.12%] [G loss: 0.834850]
(32, 10)
9967 [D loss: 0.743590, acc.: 42.19%] [G loss: 0.888058]
(32, 10)
9968 [D loss: 0.622290, acc.: 62.50%] [G loss: 0.826867]
(32, 10)
9969 [D loss: 0.631398, acc.: 68.75%] [G loss: 0.853621]
(32, 10)
9970 [D lo

(32, 10)
10083 [D loss: 0.645624, acc.: 57.81%] [G loss: 0.955242]
(32, 10)
10084 [D loss: 0.650065, acc.: 62.50%] [G loss: 0.855073]
(32, 10)
10085 [D loss: 0.634807, acc.: 64.06%] [G loss: 0.882341]
(32, 10)
10086 [D loss: 0.657956, acc.: 60.94%] [G loss: 0.876586]
(32, 10)
10087 [D loss: 0.645415, acc.: 60.94%] [G loss: 0.930436]
(32, 10)
10088 [D loss: 0.602538, acc.: 68.75%] [G loss: 0.952869]
(32, 10)
10089 [D loss: 0.673244, acc.: 57.81%] [G loss: 0.919062]
(32, 10)
10090 [D loss: 0.659400, acc.: 57.81%] [G loss: 0.880454]
(32, 10)
10091 [D loss: 0.681465, acc.: 53.12%] [G loss: 0.938050]
(32, 10)
10092 [D loss: 0.638246, acc.: 68.75%] [G loss: 0.996968]
(32, 10)
10093 [D loss: 0.564150, acc.: 73.44%] [G loss: 1.072971]
(32, 10)
10094 [D loss: 0.734276, acc.: 56.25%] [G loss: 0.961071]
(32, 10)
10095 [D loss: 0.638055, acc.: 65.62%] [G loss: 0.879455]
(32, 10)
10096 [D loss: 0.624869, acc.: 68.75%] [G loss: 0.934397]
(32, 10)
10097 [D loss: 0.704163, acc.: 56.25%] [G loss: 0.986

10207 [D loss: 0.673056, acc.: 48.44%] [G loss: 0.840271]
(32, 10)
10208 [D loss: 0.642487, acc.: 64.06%] [G loss: 0.827956]
(32, 10)
10209 [D loss: 0.674451, acc.: 59.38%] [G loss: 0.845639]
(32, 10)
10210 [D loss: 0.692548, acc.: 60.94%] [G loss: 0.909757]
(32, 10)
10211 [D loss: 0.618289, acc.: 60.94%] [G loss: 0.915896]
(32, 10)
10212 [D loss: 0.700974, acc.: 53.12%] [G loss: 0.890142]
(32, 10)
10213 [D loss: 0.682320, acc.: 56.25%] [G loss: 0.877403]
(32, 10)
10214 [D loss: 0.650676, acc.: 65.62%] [G loss: 0.827281]
(32, 10)
10215 [D loss: 0.635891, acc.: 64.06%] [G loss: 0.864985]
(32, 10)
10216 [D loss: 0.614324, acc.: 65.62%] [G loss: 0.818167]
(32, 10)
10217 [D loss: 0.647673, acc.: 62.50%] [G loss: 0.845510]
(32, 10)
10218 [D loss: 0.661554, acc.: 54.69%] [G loss: 0.879558]
(32, 10)
10219 [D loss: 0.684601, acc.: 57.81%] [G loss: 0.984356]
(32, 10)
10220 [D loss: 0.622936, acc.: 67.19%] [G loss: 0.966246]
(32, 10)
10221 [D loss: 0.669613, acc.: 64.06%] [G loss: 1.004367]
(32,

10334 [D loss: 0.694625, acc.: 60.94%] [G loss: 0.861596]
(32, 10)
10335 [D loss: 0.714742, acc.: 40.62%] [G loss: 0.944767]
(32, 10)
10336 [D loss: 0.690842, acc.: 54.69%] [G loss: 0.876008]
(32, 10)
10337 [D loss: 0.651684, acc.: 57.81%] [G loss: 0.881552]
(32, 10)
10338 [D loss: 0.620158, acc.: 65.62%] [G loss: 0.900293]
(32, 10)
10339 [D loss: 0.600579, acc.: 67.19%] [G loss: 0.876176]
(32, 10)
10340 [D loss: 0.633814, acc.: 64.06%] [G loss: 0.961375]
(32, 10)
10341 [D loss: 0.674080, acc.: 59.38%] [G loss: 0.865118]
(32, 10)
10342 [D loss: 0.646798, acc.: 65.62%] [G loss: 0.919500]
(32, 10)
10343 [D loss: 0.674938, acc.: 60.94%] [G loss: 0.842201]
(32, 10)
10344 [D loss: 0.660983, acc.: 53.12%] [G loss: 0.920877]
(32, 10)
10345 [D loss: 0.680186, acc.: 56.25%] [G loss: 0.910926]
(32, 10)
10346 [D loss: 0.653951, acc.: 60.94%] [G loss: 0.871921]
(32, 10)
10347 [D loss: 0.675700, acc.: 53.12%] [G loss: 0.959400]
(32, 10)
10348 [D loss: 0.649996, acc.: 59.38%] [G loss: 0.917980]
(32,

KeyboardInterrupt: 

In [60]:
# 自分でラベルを設定して作らせる
noise = np.random.normal(0, 1, (10, 100))
labels = np.zeros((10,10))
labels = np.zeros((10,10))
y = np.array([3,5,9,0,8,5,2,7,4,6]).reshape(10,1)
for i, v in enumerate(y):
    labels[i][v]=1
gen_imgs = gan.generator.predict([noise, labels])
gen_imgs = 0.5 * gen_imgs + 0.5

fig, axs = plt.subplots(2, 5)
cnt = 0
for i in range(2):
    for j in range(5):
        axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
        axs[i,j].axis('off')
        cnt += 1
fig.savefig("images2/%d.png" % cnt)
plt.close()